<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# _Transfer learning_<a id="top"></a>

<i><small>Autor: Alberto Díaz Álvarez<br>Última actualización: 2023-03-28</small></i></div>
                                                  

***

## Introducción

En este notebook vamos a trabajar con el concepto de _Transfer Learning_. La idea tras este es "vamos a aprovechar los conocimientos aprendidos en un modelo para el entrenamiento de otro modelo".

Siendo un poco más específicos, el proceso consiste en hacer uso de una red neuronal previamente entrenada con un buen rendimiento en un conjunto de datos más amplio, usándola como base sobre la que crear un nuevo modelo que aproveche la exactitud de esa red anterior para una nueva tarea. La "intuición" detrás de esto es que, como las primeras capas se encargan de características determinadas (en nuestro ejemplo, de características propias de imágenes), un problema que también trate con este tipo de características usará las mismas o muy parecidas (en nuestro ejemplo bordes, manchas, etc.)

## Objetivos

Aprenderemos a salvar y cargar modelos, y a usarlos parcialmente para crear modelos basados en otros ya entrenados usando la técnicas del _transfer learning_.

## Imports y configuración

A continuación importaremos las librerías que se usarán a lo largo del notebook.

In [ ]:
import emnist
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

Asímismo, configuramos algunos parámetros para adecuar la presentación gráfica.

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

***

## Extracción de características vs. ajuste fino (fine-tuning)

Existen dos extremos a la hora de usar el _Transfer Learning_; en uno de ellos, partimos de una red preentrenada, pero permitimos que se modifiquen algunos de los pesos (normalmente la última capa o las últimas capas). Se denomina "ajuste fino" o _fine-tuning_ porque estamos ajustando ligeramente los pesos de la red preentrenada a la nueva tarea. Normalmente entrenamos una red de este tipo con una tasa de aprendizaje más baja de lo normal, ya que esperamos que las características ya sean relativamente buenas y no sea necesario cambiarlas demasiado.

En el otro extremo, consiste en tomar la red preentrenada y congelar totalmente los pesos, utilizando una de sus capas ocultas (normalmente la última) como extractor de características y, por tanto, como entrada a una red neuronal más pequeña.

## Salvando y cargando nuestro modelo

En este ejemplo vamos a usar primero la red neuronal que usamos en el primer ejercicio para resolver el problema MNIST. La vamos a salvar y a cargar para volver a entrenarla, pero esta vez con los pesos ya cargados. Esto es únicamente una muestra, para más información por favor visitar [Save and load Keras models](https://www.tensorflow.org/guide/keras/save\_and\_serialize?hl=sl), donde se muestran distintas alternativas de salvado y cargado.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print(f'Training shape: {x_train.shape} input, {y_train.shape} output')
print(f'Test shape:     {x_test.shape} input, {y_test.shape} output')

En esta ocasión no vamos a hacer una codificación _one-hot_ en la salida. Existe un método para calcular el _loss_ análogo a `categorical_crossentropy` denominado `sparse_categorical_crossentropy` que esencialmente hace lo mismo. La única diferencia es el formato en el que se representa la salida.

Si esta es en formato _one-hot_, es necesario categorical_crossentropy, y si es un valor entero, `sparse_categorical_crossentropy`. No tiene más. El uso depende totalmente de cómo cargue el conjunto de datos. Una ventaja de utilizar la `sparse_categorical_crossentropy` es que ahorra memoria al utilizar un único número entero para una clase, en lugar de un vector completo.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics = ['sparse_categorical_accuracy'])
model.summary()
history = model.fit(x_train, y_train, epochs=100, batch_size=len(x_train), validation_split=0.1)

Veamos la evolución del entrenamiento gráficamente.

In [ ]:
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

Ahora vamos a salvar el modelo. Evaluaremos contra el conjunto de test antes de salvarlo y después de cargarlo para asegurarnos de que es el mismo.

In [ ]:
# Sacamos medidas
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Original model: {loss} loss, {accuracy} accuracy')
# Salvamos el modelo
model.save('tmp/supermodel.h5')
# Cargamos el modelo
model2 = tf.keras.models.load_model('tmp/supermodel.h5')
# Sacamos las medidas del modelo cargado
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Lodaded model:  {loss} loss, {accuracy} accuracy')

Lo bueno es que ahora podemos seguir entrenando el modelo en el punto que lo dejamos.

In [ ]:
history = model2.fit(x_train, y_train, epochs=100, batch_size=len(x_train), validation_split=0.1)

Podemos examinar la tendencia de esta fase de entrenamiento para comprobar que, efectivamente, comienza en el punto que se quedó en el anterior.

In [ ]:
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

Ahora, vamos a usar nuestro modelo para intentar reconocer no sólo números, sino también letras (un poco pretencioso, sí, pero es para aprender a usar nuestros modelos con _transfer learning_). Para ello nos apoyaremos en el conjunto [https://www.nist.gov/itl/products-and-services/emnist-dataset](EMNIST (Extended MNIST)) y de un _wrapper_ llamado `emnist` (`pip install emnist`) para no tener que descargar el dataset a mano.

Una vez lo tenemos instalado, haremos uso del _dataset_ con las clases balanceadas: 

In [ ]:
x_train_emnist, y_train_emnist = emnist.extract_training_samples('balanced')
x_test_emnist, y_test_emnist = emnist.extract_test_samples('balanced')

x_train_emnist = x_train_emnist / 255
x_test_emnist = x_test_emnist / 255

print(f'Training shape: {x_train_emnist.shape} input, {y_train_emnist.shape} output')
print(f'Test shape:     {x_test_emnist.shape}  input, {y_test_emnist.shape} output')

Los ejemplos tienen la siguiente forma:

In [ ]:
plt.imshow(x_train_emnist[0], cmap='hot');

Y sus etiquetas correspondientes:

In [ ]:
print(y_train_emnist[0])

Lo que haremos será cargar nuestro modelo previamente salvado y usaremos sus primeras capas sin modificar. Únicamente cambiaremos la última capa para que clasifique nuestros ejemplos, que son bastantes más.

Ésta será la única capa que entrenaremos. Esto se hace con la suposición de que las primeras capas de un modelo extraen o aprenden las características relevantes que hacen a los ejemplos únicos, y que las últimas aprenden a inferir a partir de estas características.

In [ ]:
model = tf.keras.models.load_model('tmp/supermodel.h5')

model_emnist = tf.keras.Sequential()
for i, layer in enumerate(model.layers[:-1]):  # ¡¡No incluimos la última!!
    model_emnist.add(layer)
    model_emnist.layers[-1].trainable = False  # Si no, entrenará los parámetros de estas capas
model_emnist.add(tf.keras.layers.Dense(47, activation='softmax'))

model_emnist.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics = ['sparse_categorical_accuracy'])
model_emnist.summary()

Viendo el resumen de la arquitectura de la red podemos observar que de todos los parámetros que existen, sólo se entrenarán 235, los correspondientes a las conexiones entre la penúltima y última capa.

In [ ]:
history = model_emnist.fit(x_train_emnist, y_train_emnist, epochs=100, batch_size=len(x_train_emnist), validation_split=0.1)

Ahora veamos cómo ha progresado la evolución del entrenamiento

In [ ]:
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

Bueno, quizá no ha sido el mejor ejemplo, ya que es modelo de partida es pequeño y poco generalista. Pero al menos ya sabemos como manipular un modelo para crear uno nuevo a partir de otro previamente entrenado. Ahora usaremos otro modelo más grande para ver cómo se comporta con el _dataset_ EMNIST.

### Usando un modelo preentrenado

Podemos hacer uso de los modelos salvados como parte de nuevos modelos. No suele ser trivial, pero no demasiado complicado ya que los propios modelos suelen proporcionar ayuda o, al menos, la descripción de la arquitectura para entender cómo realizarlos.

Disponemos de múltiples modelos con los que trabajar. Sólo en Keras existen decenas de modelos preentrenados listos para descargar desde la API `applications`. Veamos un ejemplo con el modelo _ResNet50_.

Para ello, necesitaremos transformar nuestras imágenes del EMNIST (en realidad arrays bidimensionales) en imágenes de 3 canales de color. Además el tamaño mínimo de imagen esperado es de $75 \times 75$, por lo que tendremos que ajustarlas también.

In [ ]:
x_train = x_train_emnist.reshape((-1, 28, 28, 1))
x_test = x_test_emnist.reshape((-1, 28, 28, 1))
x_train, x_test = tf.image.resize(x_train, (32, 32)), tf.image.resize(x_test, (32, 32))
x_train, x_test = tf.image.grayscale_to_rgb(x_train), tf.image.grayscale_to_rgb(x_test)
y_train, y_test = y_train_emnist, y_test_emnist

print(f'Training shape: {x_train.shape} input, {y_train.shape} output')
print(f'Test shape:     {x_test.shape}  input, {y_test.shape} output')

Ahora, al igual que hemos hecho antes, cargaremos el modelo ResNet50 sin incluir la última capa (argumento `include_top` a `False`). Además especificaremos que **no** queremos entrenar el modelo precargado. Los ingenieros de Microsoft han empleado tiempo y máquinas como para que estos modelos estén bastante bien entrenados.

In [ ]:
pretrained_model = tf.keras.applications.ResNet50(input_shape=[32, 32, 3], include_top=False)
pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(47, activation='softmax')
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics = ['sparse_categorical_accuracy'])
model.summary()

Ahora sólo nos quedaría entrenar. Afortunadamente, de los más de 23 millones de parámetros, sólo ajustaremos algo más de 96000, así que genial.

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=1024, validation_split=0.1)

Lento, ¿eh? Bueno, aunque nos hemos evitado entrenar millones de parámetros, lo que no hemos podido evitar es la inferencia, y esta suele costar.

Veamos cómo ha evolucionado el entrenamiento.

In [ ]:
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

## ¿De dónde obtener modelos preentrenados?

A fecha de hoy (28 de marzo de 2023) existen más $38$ modelos preentrenados disponibles en Keras a través de la API `applications`. Al descargarlo, los pesos se descargarán automáticamente en el directorio `~/.keras/models/`. Desafortunadamente, todos los modelos de la API hasta la fecha se usan para imágenes.

Tenemos disponibles no obstante más fuentes de modelos preentrenados.

### [TensorFlow Hub](https://tfhub.dev/)

Como no  podía ser de otro modo, existe un "Hub" para modelos de TensorFlow. Un ejemplo de instanciación puede ser el siguiente:

```python
import tensorflow_hub as hub
  
model = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1',
        trainable=False
    ),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
```

El API de TensorFlow Hub está disponible a traves de pip: `pip install --upgrade tensorflow_hub`

### Embeddings

Los veremos más adelante en Procesamiento de Lenguaje Natural (NLP), pero para que quede dicho, existen alguos proyectos independientes lo suficientemente famosos para disponer de su propio sitio de descarga. Este es el caso de los _embeddings_, como por ejemplo:

* [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/)
* [Word2vec](https://code.google.com/archive/p/word2vec/)
* [fastText](https://fasttext.cc/docs/en/english-vectors.html)

En el tema de NLP veremos cómo usar algunos de estos _embeddings_.

### [Hugging face](https://huggingface.co/)

Hugging Face es una empresa con sede en los Estados Unidos que desarrolla herramientas para la creación de aplicaciones basadas en aprendizaje automático. Son los desarrolladores de la biblioteca `transformers`, la cual se usa extensamente para aplicaciones de NLP.

Su plataforma permite a los usuarios compartir modelos y conjuntos de datos de aprendizaje automático. En la actualidad disponen de miles de modelo preentrenados para realizar tareas de todo tipo:

- **Visión artificial**, como clasificación de imágenes o de vídeo, segmentación o detección de objetos.
- **NLP**, como análisis de sentimiento, generación de texto o traducción.
- **Audio**, como reconocimiento del habla, clasificación o generación de audio.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>